In [1]:
import datetime
test_start_date = datetime.datetime.now() - datetime.timedelta(days = 2)
test_end_date = datetime.datetime.now() # datetime.timedelta(days = 1)

from newsapi import NewsAPI
key = '96af62a035db45bda517a9ca62a25ac3'
params = {}
api = NewsAPI(key)
sources = api.sources(params)
articles = api.articles(sources[0]['id'], params)
import sys, csv, json
#reload(sys)
#sys.setdefaultencoding('utf8')
import requests
"""
About:
Python wrapper for the New York Times Archive API 
https://developer.nytimes.com/article_search_v2.json
"""
class APIKeyException(Exception):
    def __init__(self, message): self.message = message 
class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message 
class ArchiveAPI(object):
    def __init__(self, key=None):
        """
        Initializes the ArchiveAPI class. Raises an exception if no API key is given.
        :param key: New York Times API Key
        """
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}' 
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise NoAPIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None,):
        """
        Calls the archive API and returns the results as a dictionary.
        :param key: Defaults to the API key used to initialize the ArchiveAPI class.
        """
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1882
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
            
        url = self.root.format(year, month, key)
        r = requests.get(url)
        return r.json()


api = ArchiveAPI('0ba6dc04a8cb44e0a890c00df88c393a')

import datetime
now = datetime.datetime.now()
currentMonthStart=datetime.date.today().replace(day=1).strftime("%Y-%m-%d")
currentMonthStart
years = [now.year]
months = [now.month]


for year in years:
    for month in months:
        mydict = api.query(year, month)
        file_str = 'thisNews' + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()
currentDateNow=datetime.datetime.now().strftime("%Y-%m-%d")   




In [2]:
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data

# Dow Jones
param = {
    'q': ".DJI", # Stock symbol (ex: "AAPL")
    'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "INDEXDJX", # Stock exchange symbol on which stock is traded (ex: "NASD")
    'p': "1M" # Period (Ex: "1Y" = 1 year)
}
# get price data (return pandas dataframe)
df = get_price_data(param)
print(df)
df.to_csv('thisPrice.csv')

                         Open      High       Low     Close     Volume
2018-03-01 02:30:00  25485.15  25576.15  25022.42  25029.20  451872251
2018-03-02 02:30:00  25024.04  25185.35  24442.56  24608.98  506279358
2018-03-03 02:30:00  24394.91  24592.46  24217.76  24538.06  437130059
2018-03-06 02:30:00  24471.31  24961.00  24387.15  24874.76  383913608
2018-03-07 02:30:00  24965.89  24995.24  24708.41  24884.12  331766021
2018-03-08 02:30:00  24758.15  24849.68  24535.12  24801.36  394070603
2018-03-09 02:30:00  24853.41  24950.49  24703.05  24895.21  327297318
2018-03-10 02:30:00  25004.89  25336.33  25004.89  25335.74  371572275
2018-03-13 01:30:00  25372.44  25449.15  25152.02  25178.61  362330023
2018-03-14 01:30:00  25257.75  25376.40  24947.50  25007.03  447875720
2018-03-15 01:30:00  25086.97  25130.12  24668.83  24758.12  356832403
2018-03-16 01:30:00  24837.29  25053.87  24753.29  24873.66  316306467
2018-03-17 01:30:00  24877.34  25031.00  24857.09  24946.51  641882326
2018-0

In [3]:
import numpy as np
import csv, json
import pandas as pd

################################################################################################
## Preparing DJIA data
# Reading DJIA index prices csv file
with open('C:/Users/Dell/OneDrive/Project/data/new/thisPrice.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    # Converting the csv file reader to a lists 
    data_list = list(spamreader)

# Separating header from the data
header = data_list[0] 
data_list = data_list[1:] 

data_list = np.asarray(data_list)

# Selecting date and close value for each day
selected_data = data_list[:, [0, 4]]



df = pd.DataFrame(data=selected_data[0:,1:],
             index=selected_data[0:,0],
                                columns=['close'],
                                        dtype='float64')



df1 = df
idx = pd.date_range(currentMonthStart, currentDateNow)
df1.index = pd.DatetimeIndex(df1.index)
df1 = df1.reindex(idx, fill_value=np.NaN)
# df1.count() # gives 2518 count
interpolated_df = df.interpolate()
interpolated_df.count() # gives 3651 count

# Removing extra date rows added in data for calculating interpolation
interpolated_df = interpolated_df[3:]

date_format = ["%Y-%m-%dT%H:%M:%SZ", "%Y-%m-%dT%H:%M:%S+%f"]
def try_parsing_date(text):
    for fmt in date_format:
        #return datetime.strptime(text, fmt)
        try:
            return datetime.strptime(text, fmt).strftime('%Y-%m-%d')
        except ValueError:
            pass
    raise ValueError('no valid date format found')


print ('### parsing date and price done##')



years = [now.year]
months = [now.month]
dict_keys = ['pub_date', 'headline'] #, 'lead_paragraph']
articles_dict = dict.fromkeys(dict_keys)
# Filtering list for type_of_material
type_of_material_list = ['blog', 'brief', 'news', 'editorial', 'op-ed', 'list','analysis']
# Filtering list for section_name
section_name_list = ['business', 'national', 'world', 'u.s.' , 'politics', 'opinion', 'tech', 'science',  'health']
news_desk_list = ['business', 'national', 'world', 'u.s.' , 'politics', 'opinion', 'tech', 'science',  'health', 'foreign']



from datetime import datetime
current_date = datetime.now().isoformat()

#years = [2015]
#months = [3]

current_article_str = ''      

## Adding article column to dataframe
interpolated_df["articles"] = ''
count_articles_filtered = 0
count_total_articles = 0
count_main_not_exist = 0               
count_unicode_error = 0     
count_attribute_error = 0   
for year in years:
    for month in months:
        file_str = 'C:/Users/Dell/OneDrive/Project/data/new/thisNews' + '.json'
        with open(file_str) as data_file:    
            NYTimes_data = json.load(data_file)
        count_total_articles = count_total_articles + len(NYTimes_data["response"]["docs"][:])
        
        print ('Opening file:')
        print (str(year) + '-' + '{:02}'.format(month))
        
        for i in range(len(NYTimes_data["response"]["docs"][:])):
            try:
                if any(substring in NYTimes_data["response"]["docs"][:][i]['type_of_material'].lower() for substring in type_of_material_list):
                    if any(substring in NYTimes_data["response"]["docs"][:][i]['section_name'].lower() for substring in section_name_list):
                        #count += 1
                        count_articles_filtered += 1
                        #print 'i: ' + str(i)
                        articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                        articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                        #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph
                        date = try_parsing_date(articles_dict['pub_date'])
                        #print 'article_dict: ' + articles_dict['headline']
                        if date == current_date:
                            current_article_str = current_article_str + '. ' + articles_dict['headline']
                        else:  
                            interpolated_df.set_value(current_date, 'articles', interpolated_df.loc[current_date, 'articles'] + '. ' + current_article_str)
                            current_date = date
                            #interpolated_df.set_value(date, 'articles', current_article_str)
                            #print str(date) + current_article_str
                            current_article_str = articles_dict['headline']
                        # For last condition in a year
                        if (date == current_date) and (i == len(NYTimes_data["response"]["docs"][:]) - 1): 
                            interpolated_df.set_value(date, 'articles', current_article_str)   
                        
             #Exception for section_name or type_of_material absent
            except AttributeError:
                #print 'attribute error'
                #print NYTimes_data["response"]["docs"][:][i]
                count_attribute_error += 1
                # If article matches news_desk_list if none section_name found
                try:
                    if any(substring in NYTimes_data["response"]["docs"][:][i]['news_desk'].lower() for substring in news_desk_list):
                            #count += 1
                            count_articles_filtered += 1
                            #print 'i: ' + str(i)
                            articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                            articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                            #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph
                            date = try_parsing_date(articles_dict['pub_date'])
                            #print 'article_dict: ' + articles_dict['headline']
                            if date == current_date:
                                current_article_str = current_article_str + '. ' + articles_dict['headline']
                            else:  
                                interpolated_df.set_value(current_date, 'articles', interpolated_df.loc[current_date, 'articles'] + '. ' + current_article_str)
                                current_date = date
                                #interpolated_df.set_value(date, 'articles', current_article_str)
                                #print str(date) + current_article_str
                                current_article_str = articles_dict['headline']
                            # For last condition in a year
                            if (date == current_date) and (i == len(NYTimes_data["response"]["docs"][:]) - 1): 
                                interpolated_df.set_value(date, 'articles', current_article_str)   
                
                except AttributeError:
                    pass
                pass
            except KeyError:
                print ('key error')
                #print NYTimes_data["response"]["docs"][:][i]
                count_main_not_exist += 1
                pass   
            except TypeError:
                print ("type error")
                #print NYTimes_data["response"]["docs"][:][i]
                count_main_not_exist += 1
                pass
         

              
print (count_articles_filtered) 
print (count_total_articles)                     
print (count_main_not_exist)
print (count_unicode_error)



## Putting all articles if no section_name or news_desk not found
for date, row in interpolated_df.T.iteritems():   
    if len(interpolated_df.loc[date, 'articles']) <= 400:
        #print interpolated_df.loc[date, 'articles']
        #print date
        month = date.month
        year = date.year
        file_str = 'C:/Users/Dell/OneDrive/Project/data/new/thisNews' + '.json'
        with open(file_str) as data_file:    
            NYTimes_data = json.load(data_file)
        count_total_articles = count_total_articles + len(NYTimes_data["response"]["docs"][:])
        interpolated_df.set_value(date.strftime('%Y-%m-%d'), 'articles', '')
        for i in range(len(NYTimes_data["response"]["docs"][:])):
            try:
                
                articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph       
                pub_date = try_parsing_date(articles_dict['pub_date'])
                #print 'article_dict: ' + articles_dict['headline']
                if date.strftime('%Y-%m-%d') == pub_date: 
                    interpolated_df.set_value(pub_date, 'articles', interpolated_df.loc[pub_date, 'articles'] + '. ' + articles_dict['headline'])  
                
            except KeyError:
                print ('key error')
                #print NYTimes_data["response"]["docs"][:][i]
                #count_main_not_exist += 1
                pass   
            except TypeError:
                print ("type error")
                #print NYTimes_data["response"]["docs"][:][i]
                #count_main_not_exist += 1
                pass


#>>> print count_articles_filtered 
#440770
#>>> print count_total_articles 
#1073132


## Filtering the whole data for a year
#filtered_data = interpolated_df.ix['2016-01-01':'2016-12-31']
#filtered_data.to_pickle('/Users/Dinesh/Documents/Project Stock predictions/data/pickled_ten_year_all.pkl')  


# Saving the data as pickle file
interpolated_df.to_pickle('pickled_para.pkl')  


# Save pandas frame in csv form
interpolated_df.to_csv('sample_interpolated_para.csv',
                       sep='\t', encoding='utf-8')



# Reading the data as pickle file
dataframe_read = pd.read_pickle('pickled_para.pkl')



### parsing date and price done##
Opening file:
2018-03
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key 

key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
